In [1]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
!pip install feedparser
!pip install newspaper3k

--2023-02-13 14:03:56--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8276509 (7,9M) [application/json]
Saving to: ‘lesechos.json’

lesechos.json       100%[===================>]   7,89M  11,3MB/s    in 0,7s    

2023-02-13 14:03:57 (11,3 MB/s) - ‘lesechos.json’ saved [8276509/8276509]

--2023-02-13 14:03:57--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
Resolving people.irisa.fr (people.irisa.fr)... 131.254.254.107
Connecting to people.irisa.fr (people.irisa.fr)|131.254.254.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347 [application/json]
Saving to: ‘sources.json’

sources.json        100%[===================>]     347  --.-KB/s    in 0s      

2023-02-13 14:03:57 (42,9 MB/s) - ‘sources.json’

In [2]:
!python3 -m spacy download fr_core_news_md

2023-02-13 14:05:07.966109: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 14:05:08.265381: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-13 14:05:08.265423: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-13 14:05:09.396963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [109]:
import spacy
nlp = spacy.load('fr_core_news_md')

## Question 1

In [1]:
from urllib.parse import urlparse

import json
f = open('lesechos.json')
data_lesechos = json.load(f)

# (a) se premunir contre le bloquage de commande
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# (b) recuperer le fichier RSS
import feedparser as fp
url = "https://www.lemonde.fr/rss/une.xml"
data = fp.parse(url)

for item in data.entries:
    print(item.title, item.published, item.link)

f = open('sources.json')
liste_rss_flux = json.load(f)
liste_rss_flux = list(liste_rss_flux.values())


liste_new_articles = []
# In lesechos.json, links are the keys but we cannot compare these links to the link we get when we parse RSS links.
# For example, a link from lesechos.json is 
# https://investir.lesechos.fr/marches/actualites/analyse-technique-toute-poursuite-de-la-consolidation-du-contrat-cac-40-sera-limitee-1995443.php?xtor=RSS-24
# While a link from an item entry is 
# https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/la-normalisation-des-ventes-de-velo-apres-le-boom-du-covid-laisse-des-traces-dans-les-comptes-de-thule-1905607
# So we need to compare only /marches/actualites/analyse-technique-toute-poursuite-de-la-consolidation-du-contrat-cac-40-sera-limitee
liste_url_article_already_in_base = [urlparse(temp).path[:-12] for temp in list(data_lesechos.keys())]
for url in liste_rss_flux:
    data_rss = fp.parse(url)
    for item in data_rss.entries:
        link = item.link
        res = urlparse(link)
        if res.path[:-8] not in liste_url_article_already_in_base:
            liste_new_articles.append(link)
    

Air India signe un contrat record avec Airbus et Boeing prévoyant l’achat de 250 avions européens et de 220 appareils américains Tue, 14 Feb 2023 12:14:09 +0100 https://www.lemonde.fr/economie/article/2023/02/14/air-india-signe-un-contrat-record-avec-airbus-et-boeing-prevoyant-l-achat-de-250-avions-europeens-et-de-220-appareils-americains_6161763_3234.html
Réforme des retraites : les inégalités présentes dans le système devraient perdurer Tue, 14 Feb 2023 10:17:06 +0100 https://www.lemonde.fr/politique/article/2023/02/14/retraites-les-inegalites-presentes-dans-le-systeme-devraient-perdurer_6161744_823448.html
Guerre en Ukraine, en direct : « Nous allons fournir à Kiev les moyens de tenir pendant la contre-offensive du printemps », assure le secrétaire à la défense américain Tue, 14 Feb 2023 13:57:44 +0100 https://www.lemonde.fr/international/live/2023/02/14/guerre-en-ukraine-en-direct-nous-allons-fournir-a-kiev-les-moyens-de-tenir-pendant-la-contre-offensive-du-printemps-assure-le-secr

## Question 2

Les balises pour le titre et le texte d'un article sur lesechos.fr ne sont pas clairement identifiables (h1 class="sc-14kwckt-6 sc-14omazk-0 jtuxZx fsUfny" pour le title par exemple). Utiliser ```newspaper``` nous simplifie la tâche : on peut accéder facilement aux élements de la page qui nous intéressent. De plus ce package est optimisé pour les articles de presse alors qu'un package comme BeautifulSoup serait plus contraignant ici.

In [111]:
import newspaper as np

url_one_article = liste_new_articles[0]

article = np.Article('https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/la-normalisation-des-ventes-de-velo-apres-le-boom-du-covid-laisse-des-traces-dans-les-comptes-de-thule-1905607')
article.download()
article.parse()

print(article.title, article.authors)
print(article.text)
print(article.top_image)

La normalisation des ventes de vélo, après le boom du Covid, laisse des traces dans les comptes de Thule []
Par Marjorie Encelot

Publié le 10 févr. 2023 à 15:31 Mis à jour le 10 févr. 2023 à 15:57

L’année 2021, pendant la crise sanitaire, a été « exceptionnelle » pour le vélo. Trop, finalement, quand à l’instar du suédois Thule Group (-17% à la Bourse de Stockholm, plus forte baisse aujourd’hui de l’indice européen Stoxx 600), on est une entreprise cotée en Bourse, qui vend des porte-vélos et des remorques pour vélo, et que la base de comparaison est historiquement élevée au moment de présenter ses comptes 2021.

Le quatrième trimestre, au cours duquel les ventes ont chuté de près de 21% à taux de change constants (+28% par rapport à 2019), a été particulièrement « challengeant », a reconnu le patron Jan Magnus Welander lors de la conférence téléphonique avec les analystes financiers, tant les trois derniers mois de 2021 avaient été « forts », aux Etats-Unis et au Canada essentiellem

## Question 3

In [113]:
from datetime import datetime

In [114]:
for url in liste_new_articles:
    article = np.Article(url)
    article.download()
    article.parse()
    temp_dict = {'title' : article.title, 'data' : article.publish_date.strftime("%m/%d/%Y, %H:%M:%S"), 'author' : article.authors, 'category' : None, 'content' : article.text, 'image_link' : article.top_image, 'image_file' : None}
    data_lesechos[url] = temp_dict


In [116]:
with open("lesechos_updated.json", "w") as outfile:
    json.dump(data_lesechos, outfile)

## Question 4

On peut télécharger l'image à partir de l'url de l'image trouvé précédemment (cf cellule précédente) puis sauvegarder cette image dans un dossier echod_db à la racine du projet.

## Question 5

We can look into social medias such as Twitter. For example, we could follow the profile of the newspaper and for an article (in case they tweet about their article) look at some statistics such as views, RT, etc...

## Question 7

**A COMPLETER**

## Question 8

In [1]:
import spacy

2023-02-14 14:55:32.987183: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-14 14:55:33.174536: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-14 14:55:33.174554: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-14 14:55:34.086550: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
import json

In [4]:
data_lesechos_updated = json.load(open('lesechos_updated.json'))
list_text = [temp['content'] for temp in list(data_lesechos_updated.values())]
nlp = spacy.load('fr_core_news_md', disable=['tagging', 'parser'])
docs = list(nlp.pipe(list_text))
list_entites_per_doc = []
for doc in docs:
    temp = []
    for entity in doc.ents:
        temp.append({'text' : str(entity), 'label' : str(entity.label_), 'start' : str(entity.start), 'end' : str(entity.end)})
    list_entites_per_doc.append(temp)


## Question 9

In [5]:
import itertools

flat_list = list(itertools.chain(*list_entites_per_doc))

In [6]:
from collections import defaultdict


label_counts = defaultdict(int)
for item in flat_list:
    label_counts[(item['text'], item['label'])] += 1

result = defaultdict(list)
for key, count in label_counts.items():
    result[key[1]].append((count, key[0]))

for label, items in result.items():
    items.sort(reverse=True)
    result[label] = items[:20]

list_PER = result['PER']
list_ORG = result['ORG']
list_LOC = result['LOC']
list_MISC = result['MISC']


In [21]:
list_PER

[(471, 'Jerome Powell'),
 (347, 'Omicron'),
 (306, 'Bloomberg'),
 (247, 'Joe Biden'),
 (246, 'Blandine Hénault'),
 (238, 'Hermès'),
 (214, 'Christine Lagarde'),
 (210, 'Marc Angrand'),
 (171, 'Laetitia Volga'),
 (169, 'Morgan Stanley'),
 (122, 'Jackson Hole'),
 (103, 'JPMorgan'),
 (99, 'Michael Hewson'),
 (98, 'Claude Chendjou'),
 (97, 'Bertrand Boucey'),
 (94, 'Pivot'),
 (88, 'Jean-Michel Bélot'),
 (85, 'Sophie Louet'),
 (84, 'Berenberg'),
 (75, 'Emmanuel Macron')]

On compte pour chaque entité (i.e. le "texte") le nombre d'occurences et on trie en fonction de ce nombre.

## Question 10

In [59]:
list_entites_per_doc[41]

[{'text': 'Londres', 'label': 'LOC', 'start': '31', 'end': '32'}]

In [7]:
def nb_co_occurence(pair_entities : tuple, types : tuple):
    first_entity, second_entity = pair_entities
    first_type, second_type = types
    count = 0
    for doc in list_entites_per_doc:
        indice_first = False
        indice_second = False
        for entity in doc:
            if entity['text'] == first_entity and entity['label'] == first_type:
                indice_first = True
            if entity['text'] == second_entity and entity['label'] == second_type:
                indice_second = True
        if indice_first and indice_second:
            count += 1
    return count

In [8]:
nb_co_occurence(('Omicron', 'Cac 40'), ('PER', 'MISC'))

111

## Question 11

In [19]:
import itertools
from itertools import permutations
 
# initialize lists
list_per = [{'text' : e[1], 'label' : 'PER'} for e in list_PER]
list_org = [{'text' : e[1], 'label' : 'ORG'} for e in list_ORG]
list_loc = [(e[1], 'LOC') for e in list_LOC]
list_misc = [(e[1], 'MISC') for e in list_MISC]
 


In [20]:
list_per

[('Jerome Powell', 'PER'),
 ('Omicron', 'PER'),
 ('Bloomberg', 'PER'),
 ('Joe Biden', 'PER'),
 ('Blandine Hénault', 'PER'),
 ('Hermès', 'PER'),
 ('Christine Lagarde', 'PER'),
 ('Marc Angrand', 'PER'),
 ('Laetitia Volga', 'PER'),
 ('Morgan Stanley', 'PER'),
 ('Jackson Hole', 'PER'),
 ('JPMorgan', 'PER'),
 ('Michael Hewson', 'PER'),
 ('Claude Chendjou', 'PER'),
 ('Bertrand Boucey', 'PER'),
 ('Pivot', 'PER'),
 ('Jean-Michel Bélot', 'PER'),
 ('Sophie Louet', 'PER'),
 ('Berenberg', 'PER'),
 ('Emmanuel Macron', 'PER')]

In [ ]:
# create empty list to store the
# combinations
unique_combinations = []
 
# Getting all permutations of list_1
# with length of list_2
permut = itertools.permutations(list_1, len(list_2))
 
# zip() is called to pair each permutation
# and shorter list element into combination
for comb in permut:
    zipped = zip(comb, list_2)
    unique_combinations.append(list(zipped))